### Examples of easyaccess queries

(Will obviously need the easyaccess credentials, which for Lucas lices in /home/secco/.desservices.ini on Midway)

In [1]:
import easyaccess
import pandas as pd

In [2]:
conn = easyaccess.connect(section="delve") #or section="delve" really

Connecting to DB ** delve ** ...


In [9]:
query_monika='''select archive_path from pfw_attempt a, proctag p where p.tag='SHEAR_TEST_COADD' and a.id=p.pfw_attempt_id;
'''
#points to tag in DECADE database that identifies test shear coadds, contains MEDS files
#web access in eg. https://decade.ncsa.illinois.edu/deca_archive/DEC/multiepoch/shear/r5765/
#(by Monika on March 4th)

In [6]:
query_chinyi="""
select i.expnum,i.ccdnum,i.pfw_attempt_id,i.filename,i.band,
a.path,
q.T_EFF, q.PSF_FWHM, q.NOBJECTS,q.ASTROM_CHI2,q.ASTROM_SIGMA,q.ASTROM_NDETS, q.FLAG, q.SKYBRIGHTNESS,
e.EXPTIME, e.NITE, e.PROPID, e.PRESSURE, e.MJD_OBS, e.TDECDEG, e.TRADEG, e.HA, e.PROGRAM
from image i, qa_summary q, file_archive_info a, exposure e
where i.pfw_attempt_id = q.pfw_attempt_id 
and a.filename = i.filename and e.expnum = q.expnum 
and i.filetype='red_immask' and i.ccdnum=22 and
e.band in ('g','r','i','z') 


-- Other Properties
and e.exptime between 30 and 350
and q.nobjects < 7.5e5 
and q.t_eff > 0.2 
and q.psf_fwhm between 0.5 and 1.5
and q.astrom_chi2 < 180
and q.astrom_sigma <= 0.1
and q.astrom_ndets > 250 
and q.flag < 20000
and q.calnac < 14


-- Remove exposures associated with DES and/or engineering
and not e.propid in (
    '2012B-0001', -- DES WIDE
    '2012B-0002', -- DES SN
    '2012B-0003', -- DES SV
    '2012B-0004', -- DECAM SV SN
    '2012B-9996', -- DES DC7
    '2012B-9997', -- DECam Test
    '2012B-9998', -- DECam Test
    '2012B-9999', -- Commissioning
    '2013A-9999', -- Engineering
    '2014A-9000'  -- Standards
)
ORDER BY expnum  ;"""

#Chin Yi's query in finalcut objects, points to `immask` files (actual single exposures)
#PSF files found in the resulting path via ../../psf/*

In [4]:
query_robert="""
select i.expnum,i.ccdnum,i.pfw_attempt_id, i.band, a.path, i.filename, fai2.path, m.filename
from image i, qa_summary q, file_archive_info a, exposure e, miscfile m, file_archive_info fai2
where i.pfw_attempt_id = q.pfw_attempt_id 
and a.filename = i.filename and e.expnum = q.expnum 
and i.filetype='red_immask' and i.ccdnum=22 
and m.filetype='psfex_model'
and m.pfw_attempt_id=q.pfw_attempt_id and m.filename=fai2.filename and m.ccdnum=i.ccdnum
and e.band in ('g','r','i','z') 


-- Other Properties
and e.exptime between 30 and 350
and q.nobjects < 7.5e5 
and q.t_eff > 0.2 
and q.psf_fwhm between 0.5 and 1.5
and q.astrom_chi2 < 180
and q.astrom_sigma <= 0.1
and q.astrom_ndets > 250 
and q.flag < 20000
and q.calnac < 14


-- Remove exposures associated with DES and/or engineering
and not e.propid in (
    '2012B-0001', -- DES WIDE
    '2012B-0002', -- DES SN
    '2012B-0003', -- DES SV
    '2012B-0004', -- DECAM SV SN
    '2012B-9996', -- DES DC7
    '2012B-9997', -- DECam Test
    '2012B-9998', -- DECam Test
    '2012B-9999', -- Commissioning
    '2013A-9999', -- Engineering
    '2014A-9000'  -- Standards
)
ORDER BY expnum  ;"""

#Robert's suggestion to query both the immask and psfex model files together,
#also removing some of the outputs from Chin Yi's query

In [3]:
query_robert_withra="""
select i.expnum,i.ccdnum, e.radeg, e.decdeg ,i.pfw_attempt_id, i.band, a.path, i.filename, fai2.path, m.filename
from image i, qa_summary q, file_archive_info a, exposure e, miscfile m, file_archive_info fai2
where i.pfw_attempt_id = q.pfw_attempt_id 
and a.filename = i.filename and e.expnum = q.expnum 
and i.filetype='red_immask' and i.ccdnum=22 
and m.filetype='psfex_model'
and m.pfw_attempt_id=q.pfw_attempt_id and m.filename=fai2.filename and m.ccdnum=i.ccdnum
and e.band in ('g','r','i','z') 


-- Other Properties
and e.exptime between 30 and 350
and q.nobjects < 7.5e5 
and q.t_eff > 0.2 
and q.psf_fwhm between 0.5 and 1.5
and q.astrom_chi2 < 180
and q.astrom_sigma <= 0.1
and q.astrom_ndets > 250 
and q.flag < 20000
and q.calnac < 14


-- Remove exposures associated with DES and/or engineering
and not e.propid in (
    '2012B-0001', -- DES WIDE
    '2012B-0002', -- DES SN
    '2012B-0003', -- DES SV
    '2012B-0004', -- DECAM SV SN
    '2012B-9996', -- DES DC7
    '2012B-9997', -- DECam Test
    '2012B-9998', -- DECam Test
    '2012B-9999', -- Commissioning
    '2013A-9999', -- Engineering
    '2014A-9000'  -- Standards
)
ORDER BY expnum  ;"""

#Robert's suggestion to query both the immask and psfex model files together,
#also removing some of the outputs from Chin Yi's query

In [7]:
result = conn.query_to_pandas(query_monika)

In [8]:
result

,ARCHIVE_PATH
0,DEC/multiepoch/shear/r5765/DES1155-3749/p10
1,DEC/multiepoch/shear/r5765/DES1156-3706/p03
2,DEC/multiepoch/shear/r5765/DES1158-3832/p03
3,DEC/multiepoch/shear/r5765/DES1158-3914/p03
4,DEC/multiepoch/shear/r5765/DES1158-3957/p03
5,DEC/multiepoch/shear/r5765/DES1158-4040/p04
6,DEC/multiepoch/shear/r5765/DES1158-4123/p03
7,DEC/multiepoch/shear/r5765/DES1158-4206/p03
8,DEC/multiepoch/shear/r5765/DES1158-4249/p03
9,DEC/multiepoch/shear/r5765/DES1159-3706/p03


In [10]:
result = conn.query_to_pandas(query_chinyi)

In [11]:
result

,EXPNUM,CCDNUM,PFW_ATTEMPT_ID,FILENAME,BAND,PATH,T_EFF,PSF_FWHM,NOBJECTS,ASTROM_CHI2,ASTROM_SIGMA,ASTROM_NDETS,FLAG,SKYBRIGHTNESS,EXPTIME,NITE,PROPID,PRESSURE,MJD_OBS,TDECDEG,TRADEG,HA,PROGRAM
0,145430,22,20926780,D00145430_z_c22_r4148p01_immasked.fits,z,DEC/finalcut/Y5A1/20121101-r4148/D00145430/p01/red/immask,0.794915,0.937,69563,57.400002,0.046496,2903,0,16.420000,30.0,20121101,2012B-3013,None,56233.202255,-29.713471,0.598754,02:51:59.91,"Synoptic grizY Observations Centered on the Young, high-Galactic Latitude Open Cluster Blanco 1"
1,145434,22,20926782,D00145434_z_c22_r4148p01_immasked.fits,z,DEC/finalcut/Y5A1/20121101-r4148/D00145434/p01/red/immask,0.685030,1.019,68041,58.014000,0.047095,2956,0,16.200001,30.0,20121101,2012B-3013,None,56233.204155,-29.793249,0.776504,02:53:59.63,"Synoptic grizY Observations Centered on the Young, high-Galactic Latitude Open Cluster Blanco 1"
2,145506,22,20956287,D00145506_i_c22_r4148p02_immasked.fits,i,DEC/finalcut/Y5A1/20121101-r4148/D00145506/p02/red/immask,0.288226,1.091,103879,54.487000,0.040197,3265,1,17.290001,110.0,20121101,2012B-3012,None,56233.263590,-4.267639,36.112754,01:58:29.36,DECam Supernovae: An End-to-End Verification Using Pan-STARRS Data and a Custom Pipeline
3,145507,22,20956288,D00145507_i_c22_r4148p02_immasked.fits,i,DEC/finalcut/Y5A1/20121101-r4148/D00145507/p02/red/immask,0.204167,1.293,100739,55.486000,0.040627,3274,10001,17.580000,110.0,20121101,2012B-3012,None,56233.265227,-4.252055,35.929004,02:01:43.76,DECam Supernovae: An End-to-End Verification Using Pan-STARRS Data and a Custom Pipeline
4,145516,22,20926784,D00145516_z_c22_r4148p01_immasked.fits,z,DEC/finalcut/Y5A1/20121101-r4148/D00145516/p01/red/immask,0.422671,1.071,57177,82.408997,0.053092,2677,0,23.850000,30.0,20121101,2012B-3013,None,56233.277420,-29.714666,0.616454,04:40:18.67,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
112113,1071518,22,21619290,D01071518_r_c22_r5843p01_immasked.fits,r,DEC/finalcut/Y5A1_taiga/20220112-r5843/D01071518/p01/red/immask,0.823104,0.763,93991,97.807999,0.080313,1568,0,3.370000,40.0,20220112,2021A-0275,780,59592.351798,12.099694,189.000167,-01:22:20.560,A Twilight Survey For Asteroids Interior to Earths Orbit
112114,1071519,22,21619291,D01071519_r_c22_r5843p01_immasked.fits,r,DEC/finalcut/Y5A1_taiga/20220112-r5843/D01071519/p01/red/immask,1.132704,0.735,112137,83.846001,0.075656,1588,0,3.250000,40.0,20220112,2021A-0275,780,59592.352592,12.299555,186.600075,-01:11:37.680,A Twilight Survey For Asteroids Interior to Earths Orbit
112115,1071520,22,21619292,D01071520_r_c22_r5843p01_immasked.fits,r,DEC/finalcut/Y5A1_taiga/20220112-r5843/D01071520/p01/red/immask,1.312080,0.706,113288,90.335999,0.078586,1363,0,3.200000,40.0,20220112,2021A-0275,780,59592.353422,12.386305,185.442242,-01:05:46.780,A Twilight Survey For Asteroids Interior to Earths Orbit
112116,1074384,22,21614124,D01074384_r_c22_r5820p01_immasked.fits,r,DEC/finalcut/Y5A1_taiga/20220120-r5820/D01074384/p01/red/immask,0.213439,1.096,94828,154.774994,0.067260,2816,1,8.470000,150.0,20220120,2019B-0219,779,59600.088486,0.000278,42.820667,02:33:15.170,Identifying Binary Supermassive Black Holes from Continued DECam Monitoring of DES-SN Quasars


In [5]:
result = conn.query_to_pandas(query_robert)

In [6]:
result

,EXPNUM,CCDNUM,PFW_ATTEMPT_ID,BAND,PATH,FILENAME,PATH,FILENAME
0,145430,22,20926780,z,DEC/finalcut/Y5A1/20121101-r4148/D00145430/p01/red/immask,D00145430_z_c22_r4148p01_immasked.fits,DEC/finalcut/Y5A1/20121101-r4148/D00145430/p01/psf,D00145430_z_c22_r4148p01_psfexcat.psf
1,145434,22,20926782,z,DEC/finalcut/Y5A1/20121101-r4148/D00145434/p01/red/immask,D00145434_z_c22_r4148p01_immasked.fits,DEC/finalcut/Y5A1/20121101-r4148/D00145434/p01/psf,D00145434_z_c22_r4148p01_psfexcat.psf
2,145506,22,20956287,i,DEC/finalcut/Y5A1/20121101-r4148/D00145506/p02/red/immask,D00145506_i_c22_r4148p02_immasked.fits,DEC/finalcut/Y5A1/20121101-r4148/D00145506/p02/psf,D00145506_i_c22_r4148p02_psfexcat.psf
3,145507,22,20956288,i,DEC/finalcut/Y5A1/20121101-r4148/D00145507/p02/red/immask,D00145507_i_c22_r4148p02_immasked.fits,DEC/finalcut/Y5A1/20121101-r4148/D00145507/p02/psf,D00145507_i_c22_r4148p02_psfexcat.psf
4,145516,22,20926784,z,DEC/finalcut/Y5A1/20121101-r4148/D00145516/p01/red/immask,D00145516_z_c22_r4148p01_immasked.fits,DEC/finalcut/Y5A1/20121101-r4148/D00145516/p01/psf,D00145516_z_c22_r4148p01_psfexcat.psf
...,...,...,...,...,...,...,...,...
112113,1071518,22,21619290,r,DEC/finalcut/Y5A1_taiga/20220112-r5843/D01071518/p01/red/immask,D01071518_r_c22_r5843p01_immasked.fits,DEC/finalcut/Y5A1_taiga/20220112-r5843/D01071518/p01/psf,D01071518_r_c22_r5843p01_psfexcat.psf
112114,1071519,22,21619291,r,DEC/finalcut/Y5A1_taiga/20220112-r5843/D01071519/p01/red/immask,D01071519_r_c22_r5843p01_immasked.fits,DEC/finalcut/Y5A1_taiga/20220112-r5843/D01071519/p01/psf,D01071519_r_c22_r5843p01_psfexcat.psf
112115,1071520,22,21619292,r,DEC/finalcut/Y5A1_taiga/20220112-r5843/D01071520/p01/red/immask,D01071520_r_c22_r5843p01_immasked.fits,DEC/finalcut/Y5A1_taiga/20220112-r5843/D01071520/p01/psf,D01071520_r_c22_r5843p01_psfexcat.psf
112116,1074384,22,21614124,r,DEC/finalcut/Y5A1_taiga/20220120-r5820/D01074384/p01/red/immask,D01074384_r_c22_r5820p01_immasked.fits,DEC/finalcut/Y5A1_taiga/20220120-r5820/D01074384/p01/psf,D01074384_r_c22_r5820p01_psfexcat.psf


In [4]:
result_withra = conn.query_to_pandas(query_robert_withra)

In [5]:
result_withra

,EXPNUM,CCDNUM,RADEG,DECDEG,PFW_ATTEMPT_ID,BAND,PATH,FILENAME,PATH,FILENAME
0,145430,22,0.582917,-29.720417,20926780,z,DEC/finalcut/Y5A1/20121101-r4148/D00145430/p01/red/immask,D00145430_z_c22_r4148p01_immasked.fits,DEC/finalcut/Y5A1/20121101-r4148/D00145430/p01/psf,D00145430_z_c22_r4148p01_psfexcat.psf
1,145434,22,0.760417,-29.800000,20926782,z,DEC/finalcut/Y5A1/20121101-r4148/D00145434/p01/red/immask,D00145434_z_c22_r4148p01_immasked.fits,DEC/finalcut/Y5A1/20121101-r4148/D00145434/p01/psf,D00145434_z_c22_r4148p01_psfexcat.psf
2,145506,22,36.080000,-4.273861,20956287,i,DEC/finalcut/Y5A1/20121101-r4148/D00145506/p02/red/immask,D00145506_i_c22_r4148p02_immasked.fits,DEC/finalcut/Y5A1/20121101-r4148/D00145506/p02/psf,D00145506_i_c22_r4148p02_psfexcat.psf
3,145507,22,35.895833,-4.258333,20956288,i,DEC/finalcut/Y5A1/20121101-r4148/D00145507/p02/red/immask,D00145507_i_c22_r4148p02_immasked.fits,DEC/finalcut/Y5A1/20121101-r4148/D00145507/p02/psf,D00145507_i_c22_r4148p02_psfexcat.psf
4,145516,22,0.582917,-29.720417,20926784,z,DEC/finalcut/Y5A1/20121101-r4148/D00145516/p01/red/immask,D00145516_z_c22_r4148p01_immasked.fits,DEC/finalcut/Y5A1/20121101-r4148/D00145516/p01/psf,D00145516_z_c22_r4148p01_psfexcat.psf
...,...,...,...,...,...,...,...,...,...,...
112113,1071518,22,189.000000,12.100000,21619290,r,DEC/finalcut/Y5A1_taiga/20220112-r5843/D01071518/p01/red/immask,D01071518_r_c22_r5843p01_immasked.fits,DEC/finalcut/Y5A1_taiga/20220112-r5843/D01071518/p01/psf,D01071518_r_c22_r5843p01_psfexcat.psf
112114,1071519,22,186.600000,12.300000,21619291,r,DEC/finalcut/Y5A1_taiga/20220112-r5843/D01071519/p01/red/immask,D01071519_r_c22_r5843p01_immasked.fits,DEC/finalcut/Y5A1_taiga/20220112-r5843/D01071519/p01/psf,D01071519_r_c22_r5843p01_psfexcat.psf
112115,1071520,22,185.442104,12.386700,21619292,r,DEC/finalcut/Y5A1_taiga/20220112-r5843/D01071520/p01/red/immask,D01071520_r_c22_r5843p01_immasked.fits,DEC/finalcut/Y5A1_taiga/20220112-r5843/D01071520/p01/psf,D01071520_r_c22_r5843p01_psfexcat.psf
112116,1074384,22,42.820004,0.000000,21614124,r,DEC/finalcut/Y5A1_taiga/20220120-r5820/D01074384/p01/red/immask,D01074384_r_c22_r5820p01_immasked.fits,DEC/finalcut/Y5A1_taiga/20220120-r5820/D01074384/p01/psf,D01074384_r_c22_r5820p01_psfexcat.psf


In [6]:
result_withra.to_csv('finalcut_query_withradec.csv')